In [1]:
# Load Larger LSTM network and generate text
import sys
import numpy as np
import re
import os
import string
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
# Mounting GDrive on Colab
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# load ascii text and covert to lowercase
filename = "/content/gdrive/My Drive/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [0]:
remove = string.punctuation
remove = remove.replace(".", "") # don't remove full stops
pattern = r"[{}]".format(remove) # create the pattern

raw_text_modified = re.sub(pattern, "", raw_text)

In [0]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text_modified)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
# summarize the loaded data
n_chars = len(raw_text_modified)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  154859
Total Vocab:  38


#Processing data - splitting by sentences, creating sequences and applying padding

In [0]:
from keras.preprocessing.sequence import pad_sequences

def process_data(data):
    #Split the data into sentences
    corpus = data.splitlines()
    #Convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        for i in range(1, len(line)):
            sequence = list(line[:i+1])
            input_sequences.append([char_to_int[char] for char in sequence])
    
    max_sequence_len = max([len(x) for x in input_sequences])
    padded_input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
    
    return padded_input_sequences, max_sequence_len

In [0]:
X, max_sequence_len = process_data(raw_text_modified)

#Preparing the Training data set

In [0]:
dataX = []
dataY = []

for i in range(len(X)):
  dataX.append(X[i][:-1])
  dataY.append(X[i][-1])

In [0]:
n_patterns = len(dataX)

In [0]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, max_sequence_len-1, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

#Defining the model

In [12]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True,dropout=0.1))
#model.add(Dropout(0.2))
model.add(LSTM(256))
#model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

W0727 13:16:19.565547 140279839074176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 13:16:19.604320 140279839074176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 13:16:19.616591 140279839074176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 13:16:19.812847 140279839074176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0727 13:16:19.827747 

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0727 13:16:29.954361 140279839074176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0727 13:16:29.988256 140279839074176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
# define the checkpoint
filepath="/content/gdrive/My Drive/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

#Training the model

In [15]:
# fit the model
model.fit(X, y, epochs=100, batch_size=256, callbacks=callbacks_list)

W0727 13:16:42.831551 140279839074176 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
148335/148335 [==============================] - 126s 852us/step - loss: 2.7619

Epoch 00001: loss improved from inf to 2.76188, saving model to /content/gdrive/My Drive/weights-improvement-01-2.7619-bigger.hdf5
Epoch 2/100
148335/148335 [==============================] - 121s 819us/step - loss: 2.4918

Epoch 00002: loss improved from 2.76188 to 2.49179, saving model to /content/gdrive/My Drive/weights-improvement-02-2.4918-bigger.hdf5
Epoch 3/100
148335/148335 [==============================] - 121s 816us/step - loss: 2.2983

Epoch 00003: loss improved from 2.49179 to 2.29832, saving model to /content/gdrive/My Drive/weights-improvement-03-2.2983-bigger.hdf5
Epoch 4/100
148335/148335 [==============================] - 120s 810us/step - loss: 2.1502

Epoch 00004: loss improved from 2.29832 to 2.15020, saving model to /content/gdrive/My Drive/weights-improvement-04-2.1502-bigger.hdf5
Epoch 5/100
148335/148335 [==============================] - 120s 806us/step - loss: 2.0404


#Loading the best weights for prediction

In [0]:
# load the network weights
filename = "/content/gdrive/My Drive/weights-improvement-100-0.3901-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

#Predicting the next 500 characters

In [25]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = list(dataX[start])
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" 






































assistance they need is critical t "


In [26]:
# generate characters
for i in range(500):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

eperbired all tvouf and sle fown onke ovngr iittelt pear thfe asd wherks ookeny sererked ofttele bnlce whed whats bfoune bnlcedd whed whats tuomoningd tuier toatdt whan harpinatce sfpple bnlce when tolan moke tuanfcost iordh ang kov fanlilt uoek toealnlg tuuftcer sf larpensy wrnes tf ltcssed iom bnice dvoed thf atkidgddle dogr trecling the atoee tous allrg tuinilid tvittiotat trining b mittle bni iet hace onke tuofesfy oedg harrtw su musely asd hopctretexlog or foectccss o wegd iu tocadle ior cr
Done.
